#### LeetCode - SQL - #183

Refer [183. Customers Who Never Order](https://leetcode.com/problems/customers-who-never-order/description/)

Write a solution to find all customers who never order anything.

Return the result table in  **any order**.

In [ ]:
customers_data = [[1, 'Joe'], [2, 'Henry'], [3, 'Sam'], [4, 'Max']]
customers_columns = ['id', 'name']
customers_schema = {'id':'Int64', 'name':'object'}

orders_data = [[1, 3], [2, 1]]
orders_columns = ['id', 'customerId']
orders_schema = {'id':'Int64', 'customerId':'Int64'}

In [ ]:
import pandas

customers_pandas_datafarme = pandas.DataFrame(data=customers_data, columns=customers_columns).astype(customers_schema)
orders_pandas_datafarme = pandas.DataFrame(data=orders_data, columns=orders_columns).astype(orders_schema)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
customers_datafarme = spark_context.createDataFrame(customers_pandas_datafarme)
customers_datafarme.printSchema()
customers_datafarme.show(5)

orders_datafarme = spark_context.createDataFrame(orders_pandas_datafarme)
orders_datafarme.printSchema()
orders_datafarme.show(5)

In [ ]:
orders_datafarme_customerId = [row.customerId for row in orders_datafarme.select("customerId").collect()]

customers_datafarme \
    .filter(~customers_datafarme.id.isin(orders_datafarme_customerId)) \
    .select(customers_datafarme.name.alias("Customers")) \
    .show()

In [ ]:
customers_datafarme.filter(customers_datafarme.id.expectAll(orders_datafarme.customerId))

#### Using Join

In [ ]:
customers_datafarme \
    .join(orders_datafarme, customers_datafarme.id == orders_datafarme.customerId, "left") \
    .filter(orders_datafarme.id.isNull()) \
    .select(customers_datafarme.name.alias("Customers")) \
    .show()

customers_datafarme \
    .join(orders_datafarme, customers_datafarme.id == orders_datafarme.customerId, "left_anti") \
    .select(customers_datafarme.name.alias("Customers")) \
    .show()